In [16]:
from scipy.integrate import odeint
from plotly.subplots import make_subplots
import numpy as np
import sympy as sy
import IPython
from IPython.display import display
import matplotlib.pyplot as plt

def displayH(a1,a2='', a3='', a4='', a5='', a6='', a7='',):
    latex_a1 = sy.latex(a1)
    latex_a2 = sy.latex(a2)
    latex_a3 = sy.latex(a3)
    latex_a4 = sy.latex(a4)
    latex_a5 = sy.latex(a5)
    latex_a6 = sy.latex(a6)
    latex_a7 = sy.latex(a7)
    display( IPython.core.display.Math(latex_a1 + latex_a2 + latex_a3 + latex_a4 + latex_a5 + latex_a6 + latex_a7)  )

# 1. Attitude Reorientation Maneuver Using Reaction Wheels (40 pts)

The spacecraft orbiting Earth (gravitational parameter, $\mu = 398600.4418 \, \text{km}^3/\text{s}^2$), has the following Earth-Centered-Inertial (ECI) position and velocity vectors at the initial time $t_0$:

$$
\mathbf{r}(t_0) = \begin{bmatrix} 7022.743 \\ 6699.3496 \\ 3.7886 \end{bmatrix} \, \text{km}, \quad \mathbf{r}'(t_0) = \begin{bmatrix} -6.2119 \\ 0.9524 \\ 4.3946 \end{bmatrix} \, \text{km/s}
$$

The spacecraft's moment of inertia matrix is:

$$
\mathbf{I} = \begin{bmatrix} 6400 & -76.4 & -25.6 \\ -76.4 & 4730 & -40 \\ -25.6 & -40 & 8160 \end{bmatrix} \, \text{kg-m}^2.
$$

The initial orientation of the spacecraft with respect to the ECI frame is defined by the quaternion vector,

$$
\mathbf{q}(t_0) = \frac{\mathbf{z}_0}{\lVert \mathbf{z}_0 \rVert}, \quad \mathbf{z}_0 = \begin{bmatrix} 0.685 & 0.695 & 0.153 & 0.153 \end{bmatrix}^T
$$

**Note:** The quaternion $\mathbf{q}$ has its scalar component as the fourth entry.  
The spacecraft's initial angular velocity is:

$$
\mathbf{\omega}(t_0) = \begin{bmatrix} 0.53 & 0.53 & 0.053 \end{bmatrix}^\top \, \text{deg/s}
$$

**Note:** The units are given in $\text{deg/s}$ and must be converted to $\text{rad/s}$ for analysis. Additionally, the $\omega_z$ component is smaller than $\omega_x$ and $\omega_y$ by a factor of 10.

At $t_0$, the reaction wheels are not spinning; therefore, the initial wheel momentum is $\mathbf{h}_w(t_0) = \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$. The commanded (desired) quaternion is the identity quaternion,

$$
\mathbf{q} = \begin{bmatrix} 0 & 0 & 0 & 1 \end{bmatrix}^T.
$$

In addition to achieving the commanded quaternion, it is desired that the spacecraft come to a complete stop, i.e., the final angular velocity and the commanded quaternion should be $\mathbf{\omega}_c = \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T \, \text{rad/s}$. For this purpose, the following quaternion-based proportional-derivative control law needs to be tested:

$$
\mathbf{L}_{rw} = -k_p \text{sign}(\delta q_4) \delta \mathbf{q}_{1:3} - k_d \mathbf{\omega}
$$

where:

- $\mathbf{L}_{rw}$ = control torque on the spacecraft due to reaction wheels  
- $k_p = 50$, the proportional control gain  
- $k_d = 150$, the derivative control gain  
- $\delta \mathbf{\omega} = \mathbf{\omega} - \mathbf{\omega}_c$, is the angular velocity error  
- $\delta \mathbf{q} = \begin{bmatrix} \delta q_1 & \delta q_2 & \delta q_3 & \delta q_4 \end{bmatrix}^T$, is the quaternion error  

In [17]:
mu = 398600.4418

rECI_0 = np.array([7022.743,6699.3496,3.7886])
vECI_0 = np.array([-6.2119,0.9524,4.3946])

wECI_0 = 180/np.pi*np.array([0.53,0.53,0.053])
qECI_0 = np.array([0.685,0.695,0.153,0.153])

I_body = np.array([[6400,-76.4,-25.6],
                   [-76.4,4730,-40],
                   [-25.6,-40,8160]])

hw_0 = np.array([0,0,0])
qc = np.array([0,0,0,1])
wc = np.array([0,0,0])

t0 = 0
dt = 5
tf = 2*3600

times = np.arange(t0,tf,0.1)
tol = 1e-12
xECI_0 = np.hstack([rECI_0,vECI_0])

In [18]:
def Translation_EOM(x,t,mu):
    A = -mu/np.linalg.norm(x[:3])**3
    return np.array([[0,0,0,1,0,0],
                     [0,0,0,0,1,0],
                     [0,0,0,0,0,1],
                     [A,0,0,0,0,0],
                     [0,A,0,0,0,0],
                     [0,0,A,0,0,0]])@x

xECI = odeint(Translation_EOM,xECI_0,times,rtol=tol,atol=tol,args=(mu,))

In [19]:
fig, axes = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i, label in enumerate(["x","y","z"]):
    axes[i].plot(times, xECI[:,i])
    axes[i].set_ylabel(f"${label}\\;[km]$")
    axes[i].grid()
axes[-1].set_xlabel("Time [s]")
fig.suptitle("Position vs Time",fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.plot(xECI[:, 0], xECI[:, 1], xECI[:, 2], color='b')
ax.set_box_aspect([1, 1, 1])
ax.set_xlabel('$x\\;[km]$')
ax.set_ylabel('$y\\;[km]$')
ax.set_zlabel('$z\\;[km]$')
max_val = np.max(np.abs(xECI[:, :3]))
ax.set_xlim([-max_val,max_val])
ax.set_ylim([-max_val,max_val])
ax.set_zlim([-max_val,max_val])
ax.set_title('Orbit in the Inertial Frame')
ax.view_init(elev=5, azim=45)
plt.show()

In [ ]:
kp = 10
kd = 150

def A(q):
    return np.array([[ q[3], -q[2],  q[1]],
                     [ q[2],  q[3], -q[0]],
                     [-q[1],  q[0],  q[3]],
                     [-q[0], -q[1], -q[2]]])

def L_sw(q,w):
    dq = np.zeros(4)
    dq[:3] = A(qc).T@q
    dq[3] = qc.T@q
    dw = w-wc
    return -kp*np.sign(dq[3])*dq[:3]-kd*dw

def EOM(x,t):
    q = x[:4]
    w = x[4:7]
    hw = x[7:]
    dq = 0.5*A(q)@w
    dw = I@(L_sw-np.cross(w,I@w))
    L_sw = -L_sw(q,w)
    dhw = L_sw-np.cross(w,hw)
    return [dq[0],
            dq[1],
            dq[2],
            dq[3],
            dwI[0]/I[0,0],
            dwI[1]/I[1,1],
            dwI[2]/I[2,2],
            dhw[0],
            dhw[1],
            dhw[2]]